# Basic tests of SAFTools + Sparseloop

## Eyeriss v2-like leader/follower skipping

In [1]:
!safinfer -h

usage: safinfer [-h] [-i DIR_IN] [-a ARCH] [-m MAP] [-p PROB] [-s SPARSEOPTS]
                [-o DIR_OUT] [-b BINDING_OUT] [-t TAXO_SCRIPT_LIB]
                [-T TOPOLOGY_OUT] [-r] [-L] [-f LOG_FILE] [-q SETTINGS]

optional arguments:
  -h, --help            show this help message and exit
  -i DIR_IN, --dir-in DIR_IN
  -a ARCH, --arch ARCH
  -m MAP, --map MAP
  -p PROB, --prob PROB
  -s SPARSEOPTS, --sparseopts SPARSEOPTS
  -o DIR_OUT, --dir-out DIR_OUT
  -b BINDING_OUT, --binding-out BINDING_OUT
  -t TAXO_SCRIPT_LIB, --taxo-script-lib TAXO_SCRIPT_LIB
  -T TOPOLOGY_OUT, --topology-out TOPOLOGY_OUT
  -r, --reconfigurable-arch
  -L, --log
  -f LOG_FILE, --log-file LOG_FILE
  -q SETTINGS, --settings SETTINGS
                        safinfer configuration file.


In [155]:
!safinfer -i src/ref_input/ -q src/ref_input/safinfer_settings.yaml -o src/ref_output/

- SAFinfer settings path: src/ref_input/safinfer_settings.yaml
logging: False
>> SAFinfer
:: Setup
:: => Done, setup
reconfigurable_arch: False
:: Taxonomic inference
- COMPLETE: microarchitecture inference
  => SUCCESS
:: => Done, taxonomic inference
<< Done, SAFinfer


In [5]:
!cat src/ref_output/new_arch.yaml.pretty

TestArchitecture(Architecture):
- Components:
  - BackingStorage(BufferStub)
  - iact_spad(BufferStub)
  - weight_spad(BufferStub)
  - psum_spad(BufferStub)
  - reg(BufferStub)
  - BackingStorage_datatype_format_uarch(FormatUarch)
  - iact_spad_datatype_format_uarch(FormatUarch)
  - weight_spad_datatype_format_uarch(FormatUarch)
  - reg_datatype_format_uarch(FormatUarch)
  - Skipping_iact_spad1_weight_spad0(SkippingUarch)
  - Skipping_weight_spad1_psum_spad1(SkippingUarch)
  - Skipping_iact_spad1_psum_spad0(SkippingUarch)
- Nets:
 - ['BackingStorage.md_out0', 'BackingStorage_datatype_format_uarch.md_in0']
 - ['BackingStorage.at_bound_in0', 'BackingStorage_datatype_format_uarch.at_bound_out0']
 - ['BackingStorage.md_out1', 'BackingStorage_datatype_format_uarch.md_in1']
 - ['BackingStorage.at_bound_in1', 'BackingStorage_datatype_format_uarch.at_bound_out1']
 - ['BackingStorage.md_out2', 'BackingStorage_datatype_format_uarch.md_in2']
 - ['BackingStorage.at_bound_in2', 'BackingStorage_data

In [6]:
!safmodel -h

usage: safmodel [-h] [-a ARCH] [-b CHAR] [-c COMP_IN] [-d RTL] [-f LOG_FILE]
                [-i DIR_IN] [-j HCL] [-k COMP_OUT] [-L] [-m MODEL_SCRIPT_LIB]
                [-T TOPOLOGY_OUT] [-o DIR_OUT] [-r ARCH_OUT] [-s SPARSEOPTS]
                [-t TAXO_SCRIPT_LIB] [-U SETTINGS] [-u SIM]

safmodel: SAF microarchitecture modeling generator.

optional arguments:
  -h, --help            show this help message and exit
  -a ARCH, --arch ARCH  Sparseloop architecture file.
  -b CHAR, --char CHAR  CSV EAT characterization table.
  -c COMP_IN, --comp-in COMP_IN
                        One or more Accelergy component library YAML
                        filenames.
  -d RTL, --rtl RTL     Path to underlying RTL files associated with CSV EAT
                        characterization table (-b/--char)
  -f LOG_FILE, --log-file LOG_FILE
                        Log filepath (requires -L/--log to enable logging).
  -i DIR_IN, --dir-in DIR_IN
                        Input files' directory. If not s

In [156]:
!safmodel -T src/ref_output/new_arch.yaml -a src/ref_input/arch.yaml -s src/ref_input/sparseopts.yaml -c src/ref_input/compound_components.yaml -U src/ref_input/safmodel_settings.yaml  -r src/ref_output/arch_w_SAF.yaml -k   src/ref_output/ -L

SAFmodel.

Parsing input files:
- netlist: src/ref_output/new_arch.yaml
- arch: src/ref_input/arch.yaml
- sparseopts: src/ref_input/sparseopts.yaml
- compound components (input): ['src/ref_input/compound_components.yaml']
- arch output path: src/ref_output/arch_w_SAF.yaml
- compound components path (output): src/ref_output/
- SAFModel settings path: src/ref_input/safmodel_settings.yaml
logging: True
>> SAFmodel
:: Setup
:: Scale inference
Building scale inference problem...
Building RegisteredEqualityComparator_bitwidth$(u) energy area latency table.: 1
- RegisteredEqualityComparator_bitwidth$(u) MSE energy error (symbolic vs numerical): 1.4802147286861898e-31
- RegisteredEqualityComparator_bitwidth$(u) RNMSE energy error (numerical vs true): 8.4 %
- RegisteredEqualityComparator_bitwidth$(u) MSE area error (symbolic vs numerical): 1.848202831402923e-24
- RegisteredEqualityComparator_bitwidth$(u) RNMSE area error (numerical vs true): 2.6 %
Building RegisteredEqualityComparator_bitwidth$

Building RegisteredAdder_bitwidth$(u) energy area latency table.: 100%|█| 7/7 [0
- RegisteredAdder_bitwidth$(u) MSE energy error (symbolic vs numerical): 8.091175612432276e-31
- RegisteredAdder_bitwidth$(u) RNMSE energy error (numerical vs true): 0.6 %
- RegisteredAdder_bitwidth$(u) MSE area error (symbolic vs numerical): 3.2609616555664098e-25
- RegisteredAdder_bitwidth$(u) RNMSE area error (numerical vs true): 0.7 %
=> Done, build.
Solving scale inference problem.
  Termination condition: Optimal
    Objective value: 131.79232363007497
=> done, solve.
:: => Done, scale inference
:: Export Accelergy models
Exporting backend modeling suite.
=> done, export modeling suite.
:: => Done, Accelergy export
<< Done, SAFmodel


In [11]:
print("Summary of MINLP solver output:\n\n")
!sed -n '/INFO:root:  Raw solver solution output:/,/INFO:root:-- Populating analytical primitive model attributes.../p' safmodel.log

Summary of MINLP solver output:


INFO:root:  Raw solver solution output:
INFO:root:    Objective: (0.5*x6*(3.39519598264422e-06*x37**3.0 - 0.0004030131448777*x37**2.0 + 0.0196580319453973*x37 + 0.0203547110119046) + 0.3333333333333333*x19*(3.39519598264422e-06*x45**3.0 - 0.0004030131448777*x45**2.0 + 0.0196580319453973*x45 + 0.0203547110119046))*(x38*(0.00166193389941796*x5**3.0 - 0.19617144255632*x5**2.0 + 18.9034258453812*x5 + 12.6194203869047) + x15*(0.00166193389941796*x32**3.0 - 0.19617144255632*x32**2.0 + 18.9034258453812*x32 + 12.6194203869047) + x7*(0.00166193389941796*x4**3.0 - 0.19617144255632*x4**2.0 + 18.9034258453812*x4 + 12.6194203869047) + x30*(0.00166193389941796*x8**3.0 - 0.19617144255632*x8**2.0 + 18.9034258453812*x8 + 12.6194203869047) + x16*(0.00166193389941796*x14**3.0 - 0.19617144255632*x14**2.0 + 18.9034258453812*x14 + 12.6194203869047) + x40*(0.00166193389941796*x29**3.0 - 0.19617144255632*x29**2.0 + 18.9034258453812*x29 + 12.6194203869047) + x9*(0.001661933899

In [14]:
print("Summary of Accelergy primitive ERT/ART tables output by SAFModel\n\n")
!sed -n '/WARNING:root:OVERVIEW ACCELERGY TABLES:/,/WARNING:root:-- => done, generating backend primitive objective data structure/p' safmodel.log

Summary of Accelergy primitive ERT/ART tables output by SAFModel


INFO:root:  Accelergy ART
INFO:root:    MetadataParser :
INFO:root:

 clock = 5ns
technology = 45nm
area_multiplier = 1.0
format = U
strategy = sentinel
coordinate_arithmetic = no_arithmetic
md_in_rw_thresh = 16.0
md_in_pr_thresh = 1.0
md_in_ww_thresh = 16.0
 = 271.661625870602 um^2

INFO:root:

 clock = 5ns
technology = 45nm
area_multiplier = 1.0
format = U
strategy = sentinel
coordinate_arithmetic = no_arithmetic
md_in_rw_thresh = 32.0
md_in_pr_thresh = 1.0
md_in_ww_thresh = 1.0
 = 31.3283367236290 um^2

INFO:root:

 clock = 5ns
technology = 45nm
area_multiplier = 1.0
format = U
strategy = sentinel
coordinate_arithmetic = no_arithmetic
md_in_rw_thresh = 4.0
md_in_pr_thresh = 1.0
md_in_ww_thresh = 4.0
 = 85.2007444570911 um^2

INFO:root:

 clock = 5ns
technology = 45nm
area_multiplier = 1.0
format = C
strategy = sentinel
coordinate_arithmetic = no_arithmetic
md_in_rw_thresh = 32.0
md_in_pr_thresh = 1.0
md_in_ww_thresh 

In [18]:
!timeloop-model -h

ERROR: Input configuration file does not end with .cfg, .yml, or .yaml


In [157]:
!timeloop-model src/ref_input/arch.yaml src/ref_input/compound_components.yaml src/ref_input/map.yaml src/ref_input/prob.yaml src/ref_input/sparseopts.yaml -o src/ref_output/

execute:/usr/local/bin/accelergy src/ref_input/arch.yaml src/ref_input/compound_components.yaml src/ref_input/map.yaml src/ref_input/prob.yaml src/ref_input/sparseopts.yaml --oprefix timeloop-model. -o src/ref_output// > timeloop-model.accelergy.log 2>&1
Utilization = 1.00 | pJ/Algorithmic-Compute =    2.906 | pJ/Compute =    7.654


In [158]:
!timeloop-model src/ref_output/arch_w_SAF.yaml src/ref_output/compound_components.yaml src/ref_input/compound_components.yaml src/ref_input/map.yaml src/ref_input/prob.yaml src/ref_input/sparseopts.yaml -o src/ref_output/

execute:/usr/local/bin/accelergy src/ref_output/arch_w_SAF.yaml src/ref_output/compound_components.yaml src/ref_input/compound_components.yaml src/ref_input/map.yaml src/ref_input/prob.yaml src/ref_input/sparseopts.yaml --oprefix timeloop-model. -o src/ref_output// > timeloop-model.accelergy.log 2>&1
Utilization = 1.00 | pJ/Algorithmic-Compute =   11.643 | pJ/Compute =   30.671


## ExTensor-like bidirectional skipping with naive two-fingered-merge-based intersection

In [6]:
!safinfer -h

usage: safinfer [-h] [-i DIR_IN] [-a ARCH] [-m MAP] [-p PROB] [-s SPARSEOPTS]
                [-o DIR_OUT] [-b BINDING_OUT] [-t TAXO_SCRIPT_LIB]
                [-T TOPOLOGY_OUT] [-r] [-L] [-f LOG_FILE] [-q SETTINGS]

optional arguments:
  -h, --help            show this help message and exit
  -i DIR_IN, --dir-in DIR_IN
  -a ARCH, --arch ARCH
  -m MAP, --map MAP
  -p PROB, --prob PROB
  -s SPARSEOPTS, --sparseopts SPARSEOPTS
  -o DIR_OUT, --dir-out DIR_OUT
  -b BINDING_OUT, --binding-out BINDING_OUT
  -t TAXO_SCRIPT_LIB, --taxo-script-lib TAXO_SCRIPT_LIB
  -T TOPOLOGY_OUT, --topology-out TOPOLOGY_OUT
  -r, --reconfigurable-arch
  -L, --log
  -f LOG_FILE, --log-file LOG_FILE
  -q SETTINGS, --settings SETTINGS
                        safinfer configuration file.


In [14]:
root_dir='src/case_studies/cross_design_impact'
proj_dir=root_dir+'/fig1_format_comparison_setup'
! echo safinfer -a {proj_dir}/arch/arch.yaml -s {proj_dir}/sparse-opt/coordinate_list.yaml -q {root_dir}/safinfer_settings.yaml -o {root_dir}/output/

safinfer -a src/case_studies/cross_design_impact/safe_case_study/arch/arch.yaml -s src/case_studies/cross_design_impact/safe_case_study/sparse-opt/coordinate_list.yaml -q src/case_studies/cross_design_impact/safinfer_settings.yaml -o src/case_studies/cross_design_impact/output/


In [15]:
!safinfer -a {proj_dir}/arch/arch.yaml -s {proj_dir}/sparse-opt/coordinate_list.yaml -q {root_dir}/safinfer_settings.yaml -o {root_dir}/output/

Traceback (most recent call last):
  File "/usr/local/bin/conda/bin/safinfer", line 33, in <module>
    sys.exit(load_entry_point('saftools', 'console_scripts', 'safinfer')())
  File "/home/saftool/src/safinfer.py", line 85, in main
    user_attributes = safio.parse_args()
  File "/home/saftool/src/core/safinfer_io.py", line 161, in parse_args
    processed_args = process_args(args) # Get full return tuple
  File "/home/saftool/src/core/safinfer_io.py", line 83, in process_args
    arch=sl_config.load_config_yaml(args.arch)
  File "/home/saftool/src/core/helper.py", line 90, in load_config_yaml
    with open(config_filename,'r') as config_fp:
FileNotFoundError: [Errno 2] No such file or directory: 'src/case_studies/cross_design_impact/safe_case_study/arch/arch.yaml'


In [9]:
!safmodel -h

usage: safmodel [-h] [-a ARCH] [-b CHAR] [-c COMP_IN] [-d RTL] [-f LOG_FILE]
                [-i DIR_IN] [-j HCL] [-k COMP_OUT] [-L] [-m MODEL_SCRIPT_LIB]
                [-T TOPOLOGY_OUT] [-o DIR_OUT] [-r ARCH_OUT] [-s SPARSEOPTS]
                [-t TAXO_SCRIPT_LIB] [-U SETTINGS] [-u SIM]

safmodel: SAF microarchitecture modeling generator.

optional arguments:
  -h, --help            show this help message and exit
  -a ARCH, --arch ARCH  Sparseloop architecture file.
  -b CHAR, --char CHAR  CSV EAT characterization table.
  -c COMP_IN, --comp-in COMP_IN
                        One or more Accelergy component library YAML
                        filenames.
  -d RTL, --rtl RTL     Path to underlying RTL files associated with CSV EAT
                        characterization table (-b/--char)
  -f LOG_FILE, --log-file LOG_FILE
                        Log filepath (requires -L/--log to enable logging).
  -i DIR_IN, --dir-in DIR_IN
                        Input files' directory. If not s

In [10]:
!echo safmodel -T {root_dir}/output/new_arch.yaml -a {proj_dir}/arch/arch.yaml -s {proj_dir}/sparse-opt/coordinate_list.yaml -c src/ref_input/compound_components.yaml -U {root_dir}/safmodel_settings.yaml  -r {root_dir}/output/arch_w_SAF.yaml -k {root_dir}/output/ -L

safmodel -T src/case_studies/cross_design_impact/output/new_arch.yaml -a src/case_studies/cross_design_impact/fig1_format_comparison_setup/arch/arch.yaml -s src/case_studies/cross_design_impact/fig1_format_comparison_setup/sparse-opt/coordinate_list.yaml -c src/ref_input/compound_components.yaml -U src/case_studies/cross_design_impact/safmodel_settings.yaml -r src/case_studies/cross_design_impact/output/arch_w_SAF.yaml -k src/case_studies/cross_design_impact/output/ -L


In [11]:
!safmodel -T {root_dir}/output/new_arch.yaml -a {proj_dir}/arch/arch.yaml -s {proj_dir}/sparse-opt/coordinate_list.yaml -c src/ref_input/compound_components.yaml -U {root_dir}/safmodel_settings.yaml  -r {root_dir}/output/arch_w_SAF.yaml -k {root_dir}/output/ -L

SAFmodel.

Parsing input files:
- netlist: src/case_studies/cross_design_impact/output/new_arch.yaml
- arch: src/case_studies/cross_design_impact/fig1_format_comparison_setup/arch/arch.yaml
- sparseopts: src/case_studies/cross_design_impact/fig1_format_comparison_setup/sparse-opt/coordinate_list.yaml
- compound components (input): ['src/ref_input/compound_components.yaml']
- arch output path: src/case_studies/cross_design_impact/output/arch_w_SAF.yaml
- compound components path (output): src/case_studies/cross_design_impact/output/
- SAFModel settings path: src/case_studies/cross_design_impact/safmodel_settings.yaml
logging: True
>> SAFmodel
:: Setup
:: Scale inference
Building scale inference problem...
Building RegisteredEqualityComparator_bitwidth$(u) energy area latency table.: 1
- RegisteredEqualityComparator_bitwidth$(u) MSE energy error (symbolic vs numerical): 1.4802147286861898e-31
- RegisteredEqualityComparator_bitwidth$(u) RNMSE energy error (numerical vs true): 8.4 %
- Regi

Constraint constr_41[None]: x69  ==  1
Constraint constr_42[None]: x53*x20  <=  x43
Constraint constr_43[None]: x54  ==  x10
Constraint constr_44[None]: x78  ==  x24
Constraint constr_45_x43_32_upper[None]: x43 - 32  <=  32*(1 - binVar_in_45_0)
Constraint constr_45_x43_32_lower[None]: -32*(1 - binVar_in_45_0)  <=  x43 - 32
Constraint constr_45_x43_1_upper[None]: x43 - 1  <=  32*(1 - binVar_in_45_1)
Constraint constr_45_x43_1_lower[None]: -32*(1 - binVar_in_45_1)  <=  x43 - 1
Constraint constr_45_x43_2_upper[None]: x43 - 2  <=  32*(1 - binVar_in_45_2)
Constraint constr_45_x43_2_lower[None]: -32*(1 - binVar_in_45_2)  <=  x43 - 2
Constraint constr_45_x43_4_upper[None]: x43 - 4  <=  32*(1 - binVar_in_45_3)
Constraint constr_45_x43_4_lower[None]: -32*(1 - binVar_in_45_3)  <=  x43 - 4
Constraint constr_45_x43_8_upper[None]: x43 - 8  <=  32*(1 - binVar_in_45_4)
Constraint constr_45_x43_8_lower[None]: -32*(1 - binVar_in_45_4)  <=  x43 - 8
Constraint constr_45_x43_16_upper[None]: x43 - 16  <=  

Constraint constr_96_x68_1_lower[None]: -64*(1 - binVar_in_96_1)  <=  x68 - 1
Constraint constr_96_x68_2_upper[None]: x68 - 2  <=  64*(1 - binVar_in_96_2)
Constraint constr_96_x68_2_lower[None]: -64*(1 - binVar_in_96_2)  <=  x68 - 2
Constraint constr_96_x68_64_upper[None]: x68 - 64  <=  64*(1 - binVar_in_96_3)
Constraint constr_96_x68_64_lower[None]: -64*(1 - binVar_in_96_3)  <=  x68 - 64
Constraint constr_96_x68_4_upper[None]: x68 - 4  <=  64*(1 - binVar_in_96_4)
Constraint constr_96_x68_4_lower[None]: -64*(1 - binVar_in_96_4)  <=  x68 - 4
Constraint constr_96_x68_8_upper[None]: x68 - 8  <=  64*(1 - binVar_in_96_5)
Constraint constr_96_x68_8_lower[None]: -64*(1 - binVar_in_96_5)  <=  x68 - 8
Constraint constr_96_x68_16_upper[None]: x68 - 16  <=  64*(1 - binVar_in_96_6)
Constraint constr_96_x68_16_lower[None]: -64*(1 - binVar_in_96_6)  <=  x68 - 16
Constraint constr_96_sum_x68[None]: binVar_in_96_0 + binVar_in_96_1 + binVar_in_96_2 + binVar_in_96_3 + binVar_in_96_4 + binVar_in_96_5 + b

In [12]:
!cat /home/saftool/src/solver/model/build_support/build2.py

'''Solver phase 1: DFS'''
from core.helper import info, warn, error
import solver.model.build_support.relations as rn
import solver.model.build_support.scale as sc
import saflib.microarchitecture.ModelRegistry as mr
import solver.model.build_support.abstraction as ab_
import core.model.CasCompat as cc_
import sympy as sp

def subs_scale_param_boundary_conds(port_with_boundary_conds,subs_port,reln_dict,transitive_closure_relns):
    for partial_reln in reln_dict[port_with_boundary_conds]:
        transitive_closure_relns['ineq'].append(subs_port+"_"+partial_reln)

def add_scale_param_eqns(root_port,dst_port,transitive_closure_relns):
    for attr_suffix in sc.sym_suffixes:
        transitive_closure_relns['eq'].append(root_port+"_"+attr_suffix + \
                                        " == " + \
                                        dst_port+"_"+attr_suffix)

def transitive_dfs_from_primitive_port(root_port,src_port,port_list,net_list,port_net_dict,port_attr_dict, \
                

In [13]:
!timeloop-model src/ref_output/arch_w_SAF.yaml src/ref_output/compound_components.yaml src/ref_input/compound_components.yaml src/ref_input/map.yaml src/ref_input/prob.yaml src/ref_input/sparseopts.yaml -o src/ref_output/

ERROR: key not found: arithmetic, at line: 0


## Safe case study

In [1]:
!safinfer -h

usage: safinfer [-h] [-i DIR_IN] [-a ARCH] [-m MAP] [-p PROB] [-s SPARSEOPTS]
                [-o DIR_OUT] [-b BINDING_OUT] [-t TAXO_SCRIPT_LIB]
                [-T TOPOLOGY_OUT] [-r] [-L] [-f LOG_FILE] [-q SETTINGS]

optional arguments:
  -h, --help            show this help message and exit
  -i DIR_IN, --dir-in DIR_IN
  -a ARCH, --arch ARCH
  -m MAP, --map MAP
  -p PROB, --prob PROB
  -s SPARSEOPTS, --sparseopts SPARSEOPTS
  -o DIR_OUT, --dir-out DIR_OUT
  -b BINDING_OUT, --binding-out BINDING_OUT
  -t TAXO_SCRIPT_LIB, --taxo-script-lib TAXO_SCRIPT_LIB
  -T TOPOLOGY_OUT, --topology-out TOPOLOGY_OUT
  -r, --reconfigurable-arch
  -L, --log
  -f LOG_FILE, --log-file LOG_FILE
  -q SETTINGS, --settings SETTINGS
                        safinfer configuration file.


In [16]:
root_dir='src/case_studies/cross_design_impact'
proj_dir=root_dir+'/safe_case_study'

In [19]:
!safinfer -i {proj_dir}/ref_input/ -q {proj_dir}/ref_input/safinfer_settings.yaml -o {proj_dir}/ref_output/

- SAFinfer settings path: src/case_studies/cross_design_impact/safe_case_study/ref_input/safinfer_settings.yaml
logging: False
>> SAFinfer
:: Setup
:: => Done, setup
reconfigurable_arch: False
:: Taxonomic inference
- COMPLETE: microarchitecture inference
  => SUCCESS
:: => Done, taxonomic inference
<< Done, SAFinfer


In [5]:
!cat src/ref_output/new_arch.yaml.pretty

TestArchitecture(Architecture):
- Components:
  - BackingStorage(BufferStub)
  - iact_spad(BufferStub)
  - weight_spad(BufferStub)
  - psum_spad(BufferStub)
  - reg(BufferStub)
  - BackingStorage_datatype_format_uarch(FormatUarch)
  - iact_spad_datatype_format_uarch(FormatUarch)
  - weight_spad_datatype_format_uarch(FormatUarch)
  - reg_datatype_format_uarch(FormatUarch)
  - Skipping_iact_spad1_weight_spad0(SkippingUarch)
  - Skipping_weight_spad1_psum_spad1(SkippingUarch)
  - Skipping_iact_spad1_psum_spad0(SkippingUarch)
- Nets:
 - ['BackingStorage.md_out0', 'BackingStorage_datatype_format_uarch.md_in0']
 - ['BackingStorage.at_bound_in0', 'BackingStorage_datatype_format_uarch.at_bound_out0']
 - ['BackingStorage.md_out1', 'BackingStorage_datatype_format_uarch.md_in1']
 - ['BackingStorage.at_bound_in1', 'BackingStorage_datatype_format_uarch.at_bound_out1']
 - ['BackingStorage.md_out2', 'BackingStorage_datatype_format_uarch.md_in2']
 - ['BackingStorage.at_bound_in2', 'BackingStorage_data

In [6]:
!safmodel -h

usage: safmodel [-h] [-a ARCH] [-b CHAR] [-c COMP_IN] [-d RTL] [-f LOG_FILE]
                [-i DIR_IN] [-j HCL] [-k COMP_OUT] [-L] [-m MODEL_SCRIPT_LIB]
                [-T TOPOLOGY_OUT] [-o DIR_OUT] [-r ARCH_OUT] [-s SPARSEOPTS]
                [-t TAXO_SCRIPT_LIB] [-U SETTINGS] [-u SIM]

safmodel: SAF microarchitecture modeling generator.

optional arguments:
  -h, --help            show this help message and exit
  -a ARCH, --arch ARCH  Sparseloop architecture file.
  -b CHAR, --char CHAR  CSV EAT characterization table.
  -c COMP_IN, --comp-in COMP_IN
                        One or more Accelergy component library YAML
                        filenames.
  -d RTL, --rtl RTL     Path to underlying RTL files associated with CSV EAT
                        characterization table (-b/--char)
  -f LOG_FILE, --log-file LOG_FILE
                        Log filepath (requires -L/--log to enable logging).
  -i DIR_IN, --dir-in DIR_IN
                        Input files' directory. If not s

In [20]:
!safmodel -T {proj_dir}/ref_output/new_arch.yaml -a {proj_dir}/ref_input/arch.yaml -s {proj_dir}/ref_input/sparseopts.yaml -c {proj_dir}/ref_input/compound_components.yaml -U {proj_dir}/ref_input/safmodel_settings.yaml  -r {proj_dir}/ref_output/arch_w_SAF.yaml -k {proj_dir}/ref_output/ -L

SAFmodel.

Parsing input files:
- netlist: src/case_studies/cross_design_impact/safe_case_study/ref_output/new_arch.yaml
- arch: src/case_studies/cross_design_impact/safe_case_study/ref_input/arch.yaml
- sparseopts: src/case_studies/cross_design_impact/safe_case_study/ref_input/sparseopts.yaml
- compound components (input): ['src/case_studies/cross_design_impact/safe_case_study/ref_input/compound_components.yaml']
- arch output path: src/case_studies/cross_design_impact/safe_case_study/ref_output/arch_w_SAF.yaml
- compound components path (output): src/case_studies/cross_design_impact/safe_case_study/ref_output/
- SAFModel settings path: src/case_studies/cross_design_impact/safe_case_study/ref_input/safmodel_settings.yaml
logging: True
>> SAFmodel
:: Setup
:: Scale inference
Building scale inference problem...
Building RegisteredEqualityComparator_bitwidth$(u) energy area latency table.: 1
- RegisteredEqualityComparator_bitwidth$(u) MSE energy error (symbolic vs numerical): 1.480214728

Building RegisteredAdder_bitwidth$(u) energy area latency table.: 100%|█| 7/7 [0
- RegisteredAdder_bitwidth$(u) MSE energy error (symbolic vs numerical): 8.091175612432276e-31
- RegisteredAdder_bitwidth$(u) RNMSE energy error (numerical vs true): 0.6 %
- RegisteredAdder_bitwidth$(u) MSE area error (symbolic vs numerical): 3.2609616555664098e-25
- RegisteredAdder_bitwidth$(u) RNMSE area error (numerical vs true): 0.7 %
Building RegisteredAdder_bitwidth$(u) energy area latency table.: 100%|█| 7/7 [0
- RegisteredAdder_bitwidth$(u) MSE energy error (symbolic vs numerical): 8.091175612432276e-31
- RegisteredAdder_bitwidth$(u) RNMSE energy error (numerical vs true): 0.6 %
- RegisteredAdder_bitwidth$(u) MSE area error (symbolic vs numerical): 3.2609616555664098e-25
- RegisteredAdder_bitwidth$(u) RNMSE area error (numerical vs true): 0.7 %


 {'TestArchitecture__Skipping_iact_spad1_psum_spad0__IntersectionLF__md_out_pw', 'TestArchitecture__BackingStorage_datatype_format_uarch__TestMetadataPars

Constraint constr_23_x272_1_lower[None]: -64*(1 - binVar_in_23_1)  <=  x272 - 1
Constraint constr_23_x272_2_upper[None]: x272 - 2  <=  64*(1 - binVar_in_23_2)
Constraint constr_23_x272_2_lower[None]: -64*(1 - binVar_in_23_2)  <=  x272 - 2
Constraint constr_23_x272_64_upper[None]: x272 - 64  <=  64*(1 - binVar_in_23_3)
Constraint constr_23_x272_64_lower[None]: -64*(1 - binVar_in_23_3)  <=  x272 - 64
Constraint constr_23_x272_4_upper[None]: x272 - 4  <=  64*(1 - binVar_in_23_4)
Constraint constr_23_x272_4_lower[None]: -64*(1 - binVar_in_23_4)  <=  x272 - 4
Constraint constr_23_x272_8_upper[None]: x272 - 8  <=  64*(1 - binVar_in_23_5)
Constraint constr_23_x272_8_lower[None]: -64*(1 - binVar_in_23_5)  <=  x272 - 8
Constraint constr_23_x272_16_upper[None]: x272 - 16  <=  64*(1 - binVar_in_23_6)
Constraint constr_23_x272_16_lower[None]: -64*(1 - binVar_in_23_6)  <=  x272 - 16
Constraint constr_23_sum_x272[None]: binVar_in_23_0 + binVar_in_23_1 + binVar_in_23_2 + binVar_in_23_3 + binVar_in_23

Constraint constr_71_x261_4_upper[None]: x261 - 4  <=  64*(1 - binVar_in_71_4)
Constraint constr_71_x261_4_lower[None]: -64*(1 - binVar_in_71_4)  <=  x261 - 4
Constraint constr_71_x261_8_upper[None]: x261 - 8  <=  64*(1 - binVar_in_71_5)
Constraint constr_71_x261_8_lower[None]: -64*(1 - binVar_in_71_5)  <=  x261 - 8
Constraint constr_71_x261_16_upper[None]: x261 - 16  <=  64*(1 - binVar_in_71_6)
Constraint constr_71_x261_16_lower[None]: -64*(1 - binVar_in_71_6)  <=  x261 - 16
Constraint constr_71_sum_x261[None]: binVar_in_71_0 + binVar_in_71_1 + binVar_in_71_2 + binVar_in_71_3 + binVar_in_71_4 + binVar_in_71_5 + binVar_in_71_6  ==  1
Constraint constr_72[None]: x269  <=  x73
Constraint constr_73[None]: x156  <=  x128
Constraint constr_74[None]: x283  <=  x129
Constraint constr_75[None]: x77*x284  ==  x154
Constraint constr_76[None]: x126  ==  1
Constraint constr_77[None]: x128*x129  <=  x73
Constraint constr_78[None]: x284  ==  x193
Constraint constr_79[None]: x154  ==  x281
Constraint

Constraint constr_119_sum_x169[None]: binVar_in_119_0 + binVar_in_119_1 + binVar_in_119_2 + binVar_in_119_3 + binVar_in_119_4 + binVar_in_119_5 + binVar_in_119_6  ==  1
Constraint constr_120[None]: x22  <=  x254
Constraint constr_121[None]: x243  <=  x142
Constraint constr_122[None]: x232  <=  x130
Constraint constr_123[None]: x147*x30  ==  x31
Constraint constr_124[None]: x41  ==  1
Constraint constr_125[None]: x142*x130  <=  x254
Constraint constr_126[None]: x30  ==  x50
Constraint constr_127[None]: x31  ==  x196
Constraint constr_128[None]: x232  ==  x190
Constraint constr_129_x254_32_upper[None]: x254 - 32  <=  32*(1 - binVar_in_129_0)
Constraint constr_129_x254_32_lower[None]: -32*(1 - binVar_in_129_0)  <=  x254 - 32
Constraint constr_129_x254_1_upper[None]: x254 - 1  <=  32*(1 - binVar_in_129_1)
Constraint constr_129_x254_1_lower[None]: -32*(1 - binVar_in_129_1)  <=  x254 - 1
Constraint constr_129_x254_2_upper[None]: x254 - 2  <=  32*(1 - binVar_in_129_2)
Constraint constr_129_x2

Constraint constr_180[None]: x37  ==  x132
Constraint constr_181[None]: x293  ==  x246
Constraint constr_182_x27_1000_upper[None]: x27 - 1000  <=  1000*(1 - binVar_in_182_0)
Constraint constr_182_x27_1000_lower[None]: -1000*(1 - binVar_in_182_0)  <=  x27 - 1000
Constraint constr_182_sum_x27[None]: binVar_in_182_0  ==  1
Constraint constr_183[None]: x61  <=  x71
Constraint constr_184[None]: x40  ==  x248
Constraint constr_185[None]: x66  ==  x110
Constraint constr_186[None]: x157  ==  x81
Constraint constr_187[None]: x210  ==  x45
Constraint constr_188[None]: x192  ==  x48
Constraint constr_189_x71_1000_upper[None]: x71 - 1000  <=  1000*(1 - binVar_in_189_0)
Constraint constr_189_x71_1000_lower[None]: -1000*(1 - binVar_in_189_0)  <=  x71 - 1000
Constraint constr_189_sum_x71[None]: binVar_in_189_0  ==  1
Constraint constr_190[None]: x227  ==  x178
Constraint constr_191[None]: x271  ==  x143
Constraint constr_192[None]: x39  ==  x247
Constraint constr_193[None]: x65  ==  x218
Constraint c

In [21]:
print("Summary of MINLP solver output:\n\n")
!sed -n '/INFO:root:  Raw solver solution output:/,/INFO:root:-- Populating analytical primitive model attributes.../p' safmodel.log

Summary of MINLP solver output:


INFO:root:  Raw solver solution output:
INFO:root:    Objective: (0.5*x117*(3.39519598264422e-06*x230**3.0 - 0.0004030131448777*x230**2.0 + 0.0196580319453973*x230 + 0.0203547110119046) + 0.3333333333333333*x141*(3.39519598264422e-06*x16**3.0 - 0.0004030131448777*x16**2.0 + 0.0196580319453973*x16 + 0.0203547110119046))*(x29*(0.00166193389941796*x250**3.0 - 0.19617144255632*x250**2.0 + 18.9034258453812*x250 + 12.6194203869047) + x165*(0.00166193389941796*x272**3.0 - 0.19617144255632*x272**2.0 + 18.9034258453812*x272 + 12.6194203869047) + x98*(0.00166193389941796*x120**3.0 - 0.19617144255632*x120**2.0 + 18.9034258453812*x120 + 12.6194203869047) + x106*(0.00166193389941796*x228**3.0 - 0.19617144255632*x228**2.0 + 18.9034258453812*x228 + 12.6194203869047) + x223*(0.00166193389941796*x167**3.0 - 0.19617144255632*x167**2.0 + 18.9034258453812*x167 + 12.6194203869047) + x144*(0.00166193389941796*x261**3.0 - 0.19617144255632*x261**2.0 + 18.9034258453812*x261 + 

In [22]:
print("Summary of Accelergy primitive ERT/ART tables output by SAFModel\n\n")
!sed -n '/WARNING:root:OVERVIEW ACCELERGY TABLES:/,/WARNING:root:-- => done, generating backend primitive objective data structure/p' safmodel.log

Summary of Accelergy primitive ERT/ART tables output by SAFModel


INFO:root:  Accelergy ART
INFO:root:    MetadataParser :
INFO:root:

 clock = 5ns
technology = 45nm
area_multiplier = 1.0
format = U
strategy = sentinel
coordinate_arithmetic = no_arithmetic
md_in_rw_thresh = 16.0
md_in_pr_thresh = 1.0
md_in_ww_thresh = 16.0
 = 271.661625870602 um^2

INFO:root:

 clock = 5ns
technology = 45nm
area_multiplier = 1.0
format = U
strategy = sentinel
coordinate_arithmetic = no_arithmetic
md_in_rw_thresh = 32.0
md_in_pr_thresh = 2.0
md_in_ww_thresh = 1.0
 = 62.6566734472580 um^2

INFO:root:

 clock = 5ns
technology = 45nm
area_multiplier = 1.0
format = U
strategy = sentinel
coordinate_arithmetic = no_arithmetic
md_in_rw_thresh = 32.0
md_in_pr_thresh = 2.0
md_in_ww_thresh = 2.0
 = 99.3097635572743 um^2

INFO:root:

 clock = 5ns
technology = 45nm
area_multiplier = 1.0
format = U
strategy = sentinel
coordinate_arithmetic = no_arithmetic
md_in_rw_thresh = 4.0
md_in_pr_thresh = 1.0
md_in_ww_thresh 

In [18]:
!timeloop-model -h

ERROR: Input configuration file does not end with .cfg, .yml, or .yaml


In [26]:
!timeloop-model {proj_dir}/ref_input/arch.yaml {proj_dir}/ref_input/compound_components.yaml {proj_dir}/ref_input/map.yaml {proj_dir}/ref_input/prob.yaml {proj_dir}/ref_input/sparseopts.yaml -o {proj_dir}/ref_output/

execute:/usr/local/bin/accelergy src/case_studies/cross_design_impact/safe_case_study/ref_input/arch.yaml src/case_studies/cross_design_impact/safe_case_study/ref_input/compound_components.yaml src/case_studies/cross_design_impact/safe_case_study/ref_input/map.yaml src/case_studies/cross_design_impact/safe_case_study/ref_input/prob.yaml src/case_studies/cross_design_impact/safe_case_study/ref_input/sparseopts.yaml --oprefix timeloop-model. -o src/case_studies/cross_design_impact/safe_case_study/ref_output// > timeloop-model.accelergy.log 2>&1
Utilization = 1.00 | pJ/Algorithmic-Compute =    2.906 | pJ/Compute =    7.654


In [25]:
!timeloop-model {proj_dir}/ref_output/arch_w_SAF.yaml {proj_dir}/ref_output/compound_components.yaml {proj_dir}/ref_input/compound_components.yaml {proj_dir}/ref_input/map.yaml {proj_dir}/ref_input/prob.yaml {proj_dir}/ref_input/sparseopts.yaml -o {proj_dir}/ref_output/

execute:/usr/local/bin/accelergy src/case_studies/cross_design_impact/safe_case_study/ref_output/arch_w_SAF.yaml src/case_studies/cross_design_impact/safe_case_study/ref_output/compound_components.yaml src/case_studies/cross_design_impact/safe_case_study/ref_input/compound_components.yaml src/case_studies/cross_design_impact/safe_case_study/ref_input/map.yaml src/case_studies/cross_design_impact/safe_case_study/ref_input/prob.yaml src/case_studies/cross_design_impact/safe_case_study/ref_input/sparseopts.yaml --oprefix timeloop-model. -o src/case_studies/cross_design_impact/safe_case_study/ref_output// > timeloop-model.accelergy.log 2>&1
Utilization = 1.00 | pJ/Algorithmic-Compute =   11.643 | pJ/Compute =   30.671
